In [1]:
#Primer paso que se realizo fue importar todas las librerias que vamos a utilizar
import pandas as pd
import json
import re
from fastapi import FastAPI
from datetime import datetime

In [2]:
#Primer arcivho csv importado
df1 = pd.read_csv(r"C:\Users\Admin\Desktop\SoyHenry\Proyectos\Individual 1\movies_dataset.csv")

C:\Users\Admin\AppData\Local\Temp\ipykernel_1908\1350539841.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv(r"C:\Users\Admin\Desktop\SoyHenry\Proyectos\Individual 1\movies_dataset.csv")


In [3]:
#Segundo csv importado
df_credits = pd.read_csv(r"C:\Users\Admin\Desktop\SoyHenry\Proyectos\credits.csv")

Proceso de ETL

In [4]:
#Primer paso para desanidar
class desanidar:
    @staticmethod
    def convertir_a_str(valor):
        if isinstance(valor, (list, dict)):
            return json.dumps(valor)
        return str(valor)
    @staticmethod
    def extraer_nombres(valor):
        pattern = r"'name': '([^']*)'"
        coincidencias = re.findall(pattern, valor)
        if len(coincidencias) > 0:
            nombre = coincidencias[0]
            return nombre
        else:
            return None

In [5]:
#Segundo paso para desanidar (Son varios)
df1['belongs_to_collection'] = df1['belongs_to_collection'].apply(desanidar.convertir_a_str).apply(desanidar.extraer_nombres)

In [6]:
df1['genres'] = df1['genres'].apply(desanidar.convertir_a_str).apply(desanidar.extraer_nombres)

In [7]:
df1['production_companies'] = df1['production_companies'].apply(desanidar.convertir_a_str).apply(desanidar.extraer_nombres)

In [8]:
df1['production_countries'] = df1['production_countries'].apply(desanidar.convertir_a_str).apply(desanidar.extraer_nombres)

In [9]:
df1['spoken_languages'] = df1['spoken_languages'].apply(desanidar.convertir_a_str).apply(desanidar.extraer_nombres)

In [10]:
df_credits['cast'] = df_credits['cast'].apply(desanidar.convertir_a_str).apply(desanidar.extraer_nombres)

In [11]:
df_credits['crew'] = df_credits['crew'].apply(desanidar.convertir_a_str).apply(desanidar.extraer_nombres)

In [ ]:
#Borro una columnas que estaban de mas
df1 = df1.dropna(subset=['release_date'])


In [ ]:
columnas_a_eliminar = ['video', 'imdb_id', 'adult', 'original_title', 'poster_path', 'homepage']
df1 = df1.drop(columns=columnas_a_eliminar)

In [13]:
# Convertir las columnas 'budget' y 'revenue' a tipo numérico
df1['budget'] = pd.to_numeric(df1['budget'], errors='coerce')
df1['revenue'] = pd.to_numeric(df1['revenue'], errors='coerce')

# Calcular el retorno de inversión y asignar el valor 0 cuando no haya datos disponibles
df1['return'] = df1['revenue'].div(df1['budget'], fill_value=0)

In [15]:
##Cambio a numerico el tipo de datos de la columna ID
df1['id'] = pd.to_numeric(df1['id'], errors='coerce')

In [ ]:
#Observo cuales son las columnas sin ID
df1[df1['id'].isnull()]

In [17]:
#Dado que solo son 3, decido eliminarlas
df1 = df1.dropna(subset=['id'])

In [18]:
#Cambio el tipo de dato de float a int de la columna, para que coincida en ambos df
df1['id'] = df1['id'].astype(int)

In [62]:
#Realizo un merge entre los dos datasets que tenemos
df= pd.merge(df1, df_credits, on='id')

In [64]:
#Cambie nombre de crew a Director
df.rename(columns={'crew':'director'}, inplace=True)


In [65]:
#Cambie nombre de cast a actors
df.rename(columns={'cast':'actors'}, inplace=True)


In [66]:
#Cambie nombre release_date
df.rename(columns={'release_date':'release_year'}, inplace=True)

In [67]:
#Borre la columna belongs_to_collection
df.drop('belongs_to_collection', axis=1, inplace=True)

In [68]:
#Guardado de precaución
df.to_csv('movies_modificado.csv', index=False)

In [234]:
#Visualizamos el dataset
df.head(2)

,budget,genres,id,overview,popularity,production_companies,production_countries,release_year,revenue,runtime,...,tagline,title,vote_average,vote_count,return,actors,director,release_month,release_day,dia_espanol
0,30000000.0,Animation,862,"Led by Woody, Andy's toys live happily in his ...",21.946943,Pixar Animation Studios,United States of America,1995-10-30,373554033.0,81.0,...,NaN,Toy Story,7.7,5415.0,12.451801,Tom Hanks,John Lasseter,octubre,30,Lunes
1,65000000.0,Adventure,8844,When siblings Judy and Peter discover an encha...,17.015539,TriStar Pictures,United States of America,1995-12-15,262797249.0,104.0,...,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,4.043035,Robin Williams,Larry J. Franco,diciembre,15,Viernes


Todos los codigos necesarios para la API

In [221]:
# Crear una instancia de FastAPI
app = FastAPI()
#Formula para Director
@app.get("/director/{director}")
def get_director(director: str):
    df1 = df[df["director"].notna() &df["director"].str.contains(director)]
    df1["ganancia"] = df1["revenue"] - df1["budget"]
    retorno_director = df1["return"].sum()
    resp = df1[["title", "release_year", "return", "budget", "ganancia"]]

    return [director, retorno_director, resp]




In [222]:
#Pruebas para ver si funcionan los codigos
get_director("Howard Deutch")

C:\Users\Admin\AppData\Local\Temp\ipykernel_1908\1971259495.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1["ganancia"] = df1["revenue"] - df1["budget"]


['Howard Deutch',
 inf,
                         title release_year    return      budget    ganancia
 2            Grumpier Old Men   1995-12-22       NaN         0.0         0.0
 1252   Some Kind of Wonderful   1987-02-27       inf         0.0  18553948.0
 1736        The Odd Couple II   1998-04-09       NaN         0.0         0.0
 3744         The Replacements   2000-08-07       NaN         0.0         0.0
 6716               Article 99   1992-03-13  0.354221  18000000.0 -11624021.0
 23164                 Gleason   2002-10-13       NaN         0.0         0.0]

In [223]:
## Formula para VOTOS
@app.get('/votos_titulo')
def votos_titulo(titulo_de_la_filmacion):
    pelicula = df[df["title"] == titulo_de_la_filmacion]
    
    if pelicula.empty:
        return "No se encontró la filmación en el dataset."
    
    cantidad_votos = pelicula["vote_count"].iloc[0]
    
    if cantidad_votos < 2000:
        return "La filmación no cumple con la cantidad mínima de valoraciones."
    
    promedio_votos = pelicula["vote_average"].iloc[0]
    mensaje = f"La película {titulo_de_la_filmacion} fue estrenada en el año {pelicula['release_year'].iloc[0]}. Cuenta con un total de {cantidad_votos} valoraciones, con un promedio de {promedio_votos}."
    
    return mensaje


In [224]:
votos_titulo('Jumanji')

'La película Jumanji fue estrenada en el año 1995-12-15 00:00:00. Cuenta con un total de 2413.0 valoraciones, con un promedio de 6.9.'

In [225]:
## Formula para Actor
@app.get('/get_actor')
def get_actor(nombre_actor):
    actor_df = df[df["actors"].str.contains(nombre_actor, case=False, na=False)]
    actor_df["retorno"] = actor_df["revenue"] - actor_df["budget"]
    
    cantidad_peliculas = len(actor_df)
    retorno_total = actor_df["retorno"].sum()
    promedio_retorno = actor_df["retorno"].mean()
    
    mensaje = f"El actor {nombre_actor} ha participado en {cantidad_peliculas} filmaciones. Ha conseguido un retorno total de {retorno_total} con un promedio de {promedio_retorno} por filmación."
    
    return mensaje

In [226]:
get_actor('Tom hanks')

C:\Users\Admin\AppData\Local\Temp\ipykernel_1908\2790984468.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  actor_df["retorno"] = actor_df["revenue"] - actor_df["budget"]


'El actor Tom hanks ha participado en 42 filmaciones. Ha conseguido un retorno total de 6324162814.0 con un promedio de 150575305.0952381 por filmación.'

In [227]:
# Formula para Score
@app.get('/score_titulo')
def score_titulo(titulo_de_la_filmacion):
    pelicula = df[df["title"] == titulo_de_la_filmacion]
    
    if pelicula.empty:
        return "No se encontró la filmación en el dataset."
    
    titulo = pelicula["title"].iloc[0]
    ano_estreno = pelicula["release_year"].iloc[0]
    score = pelicula["popularity"].iloc[0]
    
    mensaje = f"La película {titulo} fue estrenada en el año {ano_estreno} con un score/popularidad de {score}."
    
    return mensaje


In [228]:
score_titulo('Toy Story')

'La película Toy Story fue estrenada en el año 1995-10-30 00:00:00 con un score/popularidad de 21.946943.'

In [229]:
#Formula para la cantidad de filmaciones por MES
@app.get('/cantidad_filmaciones_mes')
def cantidad_filmaciones_mes(mes):
    mes = mes.lower()
    peliculas_mes = df[df["release_month"] == mes]
    cantidad = len(peliculas_mes)
    
    mensaje = f"{cantidad} cantidad de películas fueron estrenadas en el mes {mes.capitalize()}."
    
    return mensaje

In [230]:
cantidad_filmaciones_mes('julio')

'2646 cantidad de películas fueron estrenadas en el mes Julio.'

In [231]:
#Formula para ver cuantas peliculas fueron estrenadas por cierto dia
@app.get('/cantidad_filmaciones_dia/{dia}')
def cantidad_filmaciones_dia(dia):
    dia = dia.lower()
    filmaciones_dia = df[df["dia_espanol"].str.lower() == dia]
    cantidad = len(filmaciones_dia)
    
    mensaje = f"{cantidad} cantidad de películas fueron estrenadas en el día {dia.capitalize()}."
    
    return mensaje

In [233]:
cantidad_filmaciones_dia('Martes')

'4644 cantidad de películas fueron estrenadas en el día Martes.'

In [ ]:

# Convertir columna a tipo datetime
df['release_year'] = pd.to_datetime(df['release_year'])

# Extraer el mes de la columna 'release_date'
df['release_month'] = df['release_year'].dt.month

# Mostrar el DataFrame resultante con la columna 'release_month'
print(df)


In [78]:
# Modificar los meses para la consigna
# Extraer el mes de la columna 'release_date'
df['release_month'] = df['release_year'].dt.month

In [80]:
#Cambio los numeros a su mes correspondiente
def convertir_mes_letras(numero_mes):
    meses = {
        1: "enero",
        2: "febrero",
        3: "marzo",
        4: "abril",
        5: "mayo",
        6: "junio",
        7: "julio",
        8: "agosto",
        9: "septiembre",
        10: "octubre",
        11: "noviembre",
        12: "diciembre"
    }
    
    if numero_mes in meses:
        return meses[numero_mes]
    else:
        return "Mes inválido"

# Supongamos que tienes un DataFrame llamado df con una columna "mes" que contiene números de mes
df["release_month"] = df["release_month"].map(convertir_mes_letras)

In [82]:
#Formula para crear dias
df['release_day'] = df['release_year'].dt.day

In [83]:
#Se obtiene el nombre de cada dia en formato texto
df["release_year"] = pd.to_datetime(df["release_year"])  

def obtener_nombre_dia(fecha):
    nombre_dia = fecha.strftime("%A")
    return nombre_dia

df["nombre_dia"] = df["release_year"].apply(obtener_nombre_dia)

In [84]:
#Pasamos los dias de Ingles a Español
diccionario_dias = {
    "Monday": "Lunes",
    "Tuesday": "Martes",
    "Wednesday": "Miercoles",
    "Thursday": "Jueves",
    "Friday": "Viernes",
    "Saturday": "Sábado",
    "Sunday": "Domingo"
}

df["nombre_dia_espanol"] = df["nombre_dia"].map(diccionario_dias)

In [85]:
#Borro una columna que me quedo de mas
df.drop('nombre_dia', axis=1, inplace=True)

In [89]:
#Borro columna llamada original_language
df.drop('original_language', axis=1, inplace=True)

In [135]:
#Cambie nombre release_date
df.rename(columns={'nombre_dia_espanol':'dia_espanol'}, inplace=True)

In [192]:
#Ultimo push para guardar el dataset listo
df.to_csv('Csv_Proyecto_Terminado.csv', index=False)